# Advent of Code 2020 in APL

Every day is comprised of three cells:
1. read input file and parse contents
2. solve the first problem
3. solve the second problem

## Day 1

In [2]:
⍝ Read file linewise, parse every line into a number
input1 ← ⍎¨⊃⎕NGET 'day1.txt' 1

In [3]:
day1_1 ← {×/⍵[⊃⍸2020⍷⍵∘.+⍵]}
day1_1 input1

436404

In [4]:
day1_2 ← {×/⍵[⊃⍸2020⍷⍵(∘.+⍣2)⍵]}
day1_2 input1

274879808

## Day 2

In [5]:
input2 ← ⊃⎕NGET 'day2.txt' 1
⍝ '1-2 l: phrase' ~> (1 2 l phrase)
parsed2 ← {' '(≠⊆⊢)('-|:' ⎕R ' ')⍵}¨ input2

In [6]:
⍝ Count occurrences of ⍵[3] in ⍵[4]:
occ ← {+/⊃⍵[3]=⍵[4]}
⍝ Check if ⍵[1] ≤ ⍺ ≤ ⍵[2]
between ← {((⍎⊃⍵[1])≤⍺)∧((⍎⊃⍵[2])≥⍺)}
⍝ Count correct passphrases:
day2_1 ← {+/(occ between ⊣)¨⍵}
day2_1 parsed2

434

In [7]:
day2_2 ← {≠/ (⊃⍵[3]) = (⊃⍵[4])[⍎¨⍵[1 2]]} ⍝ Check if the phrase is correct
+/day2_2¨ parsed2

509

## Day 3

In [8]:
input3 ← ⎕NGET 'day3.txt' 1
parsed3 ← ((≢,(≢⊃))⍴∊)⊃input3 ⍝ Parse input3 into a matrix of correct dimensions

In [9]:
day3_1 ← {⊃+⌿ (3×¯1+⍳≢⍵) ⌽ '#'=⍵}
day3_1 parsed3

184

In [10]:
⍝ The logic is mostly the same, but with variable sloping
day3_2 ← {⊃+⌿ (⍺×¯1+⍳≢⍵) ⌽ '#'=⍵}
⍝ Special case ⍺=÷n: take every n-th line and shift the rows 1-wise 
preproc ← {((≢⍵)⍴1,((⍺-1)⍴0))⌿⍵}
⍝ Format number in non-exponential notation
'I10' ⎕FMT (1 day3_2 2 preproc parsed3)×{1 3 5 7 ×.day3_2 ⍵} ⊂parsed3

2431272960

## Day 4

For problem one, if input is the file as one string, we want something like the following (in a Rusty syntax):
    1. let requiredFields = ['byr' 'iyr' 'eyr' 'hgt' 'hcl' 'ecl' 'pid'];
    2. let solution = input : String
        // Parsing part:
        .splitAt emptyLine : List String
        .map(|xs| xs.joinWith(' ')) : List String
        .map(words) : List (List String)
        .map∘map(|xs| xs.splitAt(':')) : List (List (String, String))
        // Logic part:
        .map∘map(first) : List (List String)
        .map(|xs| isSubset requiredFields xs) : List Bool
        .count(True);

In [11]:
input4 ← ⊃⎕NGET 'day4.txt' 0
nl ← (⎕UCS 10)
⍝ transform string into each line containing one key:value pair and a '-' between records
normalize ← {('^$'⎕R'-') (' ' ⎕R '\n') ⍵}
splitAt ← {⍺(≠⊆⊢)⍵}
⍝ Like splitAt, but checks for ∊ instead of =
⍝ Must start with a divider, else the first partition element is lost. :(
partitionAt ← {1↓¨⍺(∊¨⊂⊢)⍵}
⍝ process key:value pairs
entry ← {':' splitAt¨ ⍵}
parsed4 ← entry¨ '-' partitionAt nl splitAt normalize nl, input4

In [153]:
day4_1 ← {7 = ≢'byr' 'iyr' 'eyr' 'hgt' 'hcl' 'ecl' 'pid' ∩ ⊃¨⍵}
+/ day4_1¨ parsed4

242

For problem two, dictionaries would be really helpful … Well it works, somehow.

In [211]:
⍝ Check all conditions for every field
⍝ Multiline definitions seem not to work. :(
chbyr ← {(⍎⊃⍵) between '1920' '2002'}
checl ← {(⊂⍵) ∊ 'amb' 'blu' 'brn' 'gry' 'grn' 'hzl' 'oth'}
cheyr ← {(⍎⊃⍵) between '2020' '2030'}
chhcl ← {'.'∊('^#[0-9a-f]{6}$' ⎕R'.') ⊃⍵} ⍝ Terrible hack, but that’s how it is
chhgt ← {(≢'^(((59|6[0-9]|7[0-6])in)|(1([5-8]\d|9[0-3])cm))$' ⎕S 1) ⍵}
chiyr ← {(⍎⊃⍵) between '2010' '2020'}
chpid ← {9=+/('^\d{9}$' ⎕S 1) ⍵}
checkall←{∧/(chbyr ⍵[1]) (checl ⊃⍵[2]) (cheyr ⍵[3]) (chhcl ⍵[4]) (chhgt ⍵[5]) (chiyr ⍵[6]) (chpid ⍵[7])}

reduceFields ← {7=≢⍵: (2⊃¨⍵)[⍋⊃¨⍵] ⋄ 8=≢⍵: (1 0, 6⍴1) / (2⊃¨⍵)[⍋⊃¨⍵]}
+/ (checkall∘reduceFields)¨ {(day4_1¨ ⍵)/⍵} parsed4

186

## Day 5

For problem one, there are three steps (not necessarily in that order):
1. translate a code into a (row, seat) number pair
2. sort all entries/take the maximum
3. calculate the seat id ← seat + 8 × row

The most resource efficient would probably be 2–1–3, because then only one code has to be translated, but it should not make too much of a difference, since all three functions have to be implementet no matter what.

### Translating

```
         F         B          F          B          B          F          F
(0, 127) → (0, 63) → (32, 63) → (32, 47) → (40, 47) → (44, 47) → (44, 45) → (44, 44)
(0, 2*7) → (+0, -2*6) → (+2*5, -0) → (+0, -2*4) → ...
```

In [253]:
input5 ← ⊃⎕NGET 'day5.txt' 1

In [289]:
translate ← +/{(2*¯1+(⌽⍳7),⌽⍳3)×⍺=⍵} ⍝ Count using B for rows, R for seats
seats ← ('B'∘translate,'R'∘translate)¨ input5
day5_1 ← {+/8 1×⍵}
⌈/day5_1¨ seats

838

In [324]:
sortedSeats ← {⍵[⍒⍵]} day5_1¨ seats
day5_2 ← {¯1+⍵[⍸1≠-⌿¯1↓[2](2, ≢⍵)⍴⍵, 1⌽⍵]} ⍝ subtract neighbors, find difference greater 1
day5_2 sortedSeats

714

## Day 6

In [357]:
input6 ← ⊃⎕NGET 'day6.txt' 0
parsed6 ← '-' partitionAt normalize nl, input6 ⍝ Reuse from day 4

In [360]:
day6_1 ← {¯1+≢∪⍵}
+/ day6_1¨ parsed6

6161

In [391]:
day6_2 ← {≢⊃∩/nl(≠⊆⊢)⍵}
+/ day6_2¨ parsed6

2971

## Day 7

In [625]:
input7 ← ⊃⎕NGET 'day7.txt' 1
parsed7 ← {(1 1 0 0, (¯4+≢⍵)⍴1 1 1 0)⊆⍵}¨ ' ' (≠⊆⊢)¨ input7

For problem one, consider a directed graph (more precise a tree), whith an edge a→b iff a∊b. Calculate the adjacency matrix A. The n-th power of A gives the n-deeply nested containment, the transitive closure is the fixed point of $A \mapsto \operatorname{sgn}(A + A \times A)$.

Thus, the problem is divided into two parts:
1. Find the transitive closure T of the adjacency matrix for the baggage graph
2. Find the index of 'shiny gold' bags and +/ the the corresponding row of T

In [626]:
parsed7_1 ← {⍵[1], 1↓¨1↓⍵}¨ parsed7
eye ← {⍵ ⍵⍴1, ⍵⍴0} ⍝ unit matrix of shape ⍵ ⍵
traclo ← {⍵∨⍵∨.∧⍵}⍣≡ ⍝ transitive closure
⍝ For every bag, in which bags is it contained? (I. e. is there an edge?)
day7_1 ←{+/ (traclo (-eye ≢⍵)+(⊂¨⊃¨⍵) ∘.∊ ⍵)[((⊃¨⍵) ⍳ ⊂'shiny' 'gold');]}
day7_1 parsed7_1

257

For problem one, we have to go the other way around. An edge a→b means that a contains b; also we have a multigraph now, meaning that each entry in A contains the number of edges given by the input.

In [1365]:
foos ← (('foo' (4 'bar')) ('bar' ('no' 'more')) ('ham' ('no' 'more')) ('spam' (1 'quux') (2 'ham')) ('quux' (1 'foo') (3 'baz')) ('baz' ('no' 'more')))
⍝ Transform empty bags into the right format;
parsed7_2 ← {'no'≡⊃2⊃⍵:⊂⊃⍵ ⋄ ⍵}¨ parsed7
len ← ≢ parsed7_2
⍝ number of contained bags:
numbers7 ← {⍎¨¨(1⊃¨¨1↓¨⍵),¨'0'} parsed7_2
⍝ names of the bags contained in each bag, but catenated à la 'shiny' 'gold' → 'shinygold'
names7 ← {∊¨¨1↓¨¨1↓¨⍵} parsed7_2

⍝ [i; j] = index or ≢parsed7_2, depending on if a bag contains a bag
indices7 ← {⊃¨⍵,¨≢⍵}(⍸¨names7 ∘.(≡¨) (⊂¨∊¨⊃¨parsed7_2)) 
⍝ [i; j] = (x y) such that (numbers7[x])[y] = adjacency[x;y]
indices7_2 ← (⍉(len len)⍴ ⍳⍴ numbers7) ,¨ indices7 ⌊ ⍉(len len)⍴ ⍴¨numbers7
⍝ The adjacency matrix (don’t ask me why):
adj ← {⊃(numbers7[⍵1])}¨ indices7_2
goldIndex ← ⍸(⊂'shinygold')⍷∊¨⊃¨parsed7_2
⍝ 9 is a magic number right now; it should be ⍳(expression) = 0, but how?
⍝ with adj(+.×⍣≡)adj we get the fixed point; we want to scan until that is reached.
+/⊃+/goldIndex ⌷¨ ((+.×⍣≡)\ 9⍴ ⊂⊃¨adj)


1038